In [1]:
%%capture
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rasterio
!pip install fiona
!pip install geopandas
!pip install -i https://test.pypi.org/simple/ gis-utils-pkg-dillhicks==0.0.3

#Importing GIS Software
from rasterio import windows
from rasterio.features import shapes
from  gis_utils import raster
import geopandas as gpd
import rasterio 
import gdal 
import os

In [2]:
import itertools
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [3]:
from tensorflow.keras.layers import InputLayer, Dense, concatenate
%load_ext tensorboard

In [4]:
#loading training data

from google.colab import drive
drive.mount('/content/drive')
 
!cp "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Training Images/Hybrid CNN/obj-dataset.zip" /content/obj-dataset.zip
!unzip "/content/obj-dataset.zip" -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/dataset/1/lr_245656.tif  
  inflating: dataset/dataset/1/lr_111099.tif  
  inflating: dataset/dataset/1/lr_171921.tif  
  inflating: dataset/dataset/1/hr_128337.tif  
  inflating: dataset/dataset/1/hr_264223.tif  
  inflating: dataset/dataset/1/hr_58317.tif  
  inflating: dataset/dataset/1/hr_99810.tif  
  inflating: dataset/dataset/1/hr_126262.tif  
  inflating: dataset/dataset/1/lr_240874.tif  
  inflating: dataset/dataset/1/hr_58190.tif  
  inflating: dataset/dataset/1/lr_47466.tif  
  inflating: dataset/dataset/1/lr_106880.tif  
  inflating: dataset/dataset/1/hr_96972.tif  
  inflating: dataset/dataset/1/lr_109200.tif  
  inflating: dataset/dataset/1/lr_109003.tif  
  inflating: dataset/dataset/1/hr_240965.tif  
  inflating: dataset/dataset/1/lr_98186.tif  
  inflating: dataset/dataset/1/hr_102133.tif  
  inflating: dataset/dataset/1/hr_70620.tif  
  inflating: dataset/dataset/1/hr_42587.tif  
  inflating: data

In [5]:
from skimage.transform import resize
from os.path import basename, dirname
from rasterio.plot import reshape_as_image
import glob
import random
from sklearn.model_selection import train_test_split

def load_images(image_dir, n_bands, image_size, p=1):

    num_images = (len(glob.glob1("/content/dataset/dataset/1","*.tif")) + len(glob.glob1("/content/dataset/dataset/0","*.tif")))/2
    
    index = 0
    labels = []
    images = [] 

    for root, dirs, files in os.walk(image_dir):
        for file in tqdm(files):
            if (file.endswith(".tif") and file.startswith("hr_")):
                file_path = os.path.join(root, file)
                label = int(basename(dirname(file_path)))
                labels.append(label)
                images.append(file_path)


    c = list(zip(images, labels))

    random.shuffle(c)

    images, labels = zip(*c)

    images = images[0:int(num_images*p)]
    labels = labels[0:int(num_images*p)]

    labels = np.asarray(labels)

    x_ids = list(range(int(num_images*p)))
    x_train_ids, x_test_ids, y_train, y_test = train_test_split(x_ids, labels, test_size = 0.2, random_state=42)

    x_hr_train = np.memmap('hr_train.memmap', dtype='float32', mode='w+', shape=(len(x_train_ids),image_size[0],image_size[1],3))
    x_hr_test = np.memmap('hr_test.memmap', dtype='float32', mode='w+', shape=(len(x_test_ids),image_size[0],image_size[1],3))

    x_lr_train = np.memmap('lr_train.memmap', dtype='float32', mode='w+', shape=(len(x_train_ids),n_bands))
    x_lr_test = np.memmap('lr_test.memmap', dtype='float32', mode='w+', shape=(len(x_test_ids),n_bands))

    for i, index in enumerate(tqdm(x_train_ids)):
        
        file_path = images[index]

        hr_image, _ = raster.load_image(file_path)
        lr_image, _ = raster.load_image(file_path.replace("hr_", "lr_"))
        
        x_hr_train[i,:,:,:] = resize(reshape_as_image(hr_image.read()), (112,112,3))
        x_lr_train[i,:] = np.squeeze(np.squeeze(reshape_as_image(lr_image.read())))
        
       
    for i, index in enumerate(tqdm(x_test_ids)):

        file_path = images[index]

        hr_image, _ = raster.load_image(file_path)
        lr_image, _ = raster.load_image(file_path.replace("hr_", "lr_"))

        x_hr_test[i,:,:,:] = resize(reshape_as_image(hr_image.read()), (112,112,3))
        x_lr_test[i,:] = np.squeeze(np.squeeze(reshape_as_image(lr_image.read())))
    



    labels_array = np.asarray(labels)
    del labels

    return x_hr_train, x_hr_test, x_lr_train, x_lr_test, y_train, y_test

In [6]:
from os.path import join

for root, dirs, files in os.walk("/content/dataset/dataset/0"):
    firstfile = sorted(files)[-1]
    test_file =  os.path.join(root, firstfile)

test_img, _ = raster.load_image(test_file)
n_bands = test_img.count

In [ ]:
x_hr_train, x_hr_test, x_lr_train, x_lr_test, y_train, y_test = load_images("dataset", n_bands, (128,128))

print(y_train.shape)
print(x_hr_train.shape)
print(x_lr_train.shape)

In [ ]:
def create_cnn(module_handle, module_name, image_size, n_2):
    model = tf.keras.Sequential([
    InputLayer(input_shape = IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable = False, name = module_name),
    Dense(50)])
    return model

In [ ]:
def create_mlp(n_bands, n_1):
    model = tf.keras.Sequential([
       tf.keras.layers.Dense(32, input_dim=n_bands)                         
    ])
    return model

In [ ]:
def show_model(model):
    tf.keras.utils.plot_model(model)
    model.summary()

In [ ]:
def create_hybrid(MODULE_HANDLE, IMAGE_SIZE, BATCH_SIZE, n_1, n_2, n_bands):
    feature_extractor = create_cnn(MODULE_HANDLE,"Resnet50V2", IMAGE_SIZE,n_2)
    mlp = create_mlp(n_bands,n_1)

    combined = concatenate([mlp.output, feature_extractor.output])
    x = Dense(2, activation="sigmoid")(combined)


    model = tf.keras.Sequential()
    model = tf.keras.models.Model(inputs=[mlp.input, feature_extractor.input], outputs=x)
   
    model.compile(
    optimizer=tf.keras.optimizers.RMSprop(), 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'])

    return model

In [ ]:
def create_base(MODULE_HANDLE, IMAGE_SIZE, BATCH_SIZE):
    model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=False),
    tf.keras.layers.Dense(2, activation="sigmoid")
    ])
    model.build((None,)+IMAGE_SIZE+(3,))
    return model


In [ ]:
print(y_train.shape)
print(y_test.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
MODULE_HANDLE = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
IMAGE_SIZE = (128,128)
BATCH_SIZE = 64

n_1 = 32
n_2 = 256

model = create_hybrid(MODULE_HANDLE, IMAGE_SIZE, BATCH_SIZE, n_1, n_2, n_bands)
history = model.fit(x = [x_lr_train, x_hr_train], y = y_train, validation_data = ([x_lr_test, x_hr_test], y_test), epochs = 30)
model.save("/content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Models/Hybrid CNN/efficientnetb0-pretrained-obj")

Epoch 1/30
12478/12478 [==============================] - 4208s 337ms/step - loss: 0.5532 - accuracy: 0.9400 - val_loss: 0.5501 - val_accuracy: 0.9488
Epoch 2/30
 1011/12478 [=>............................] - ETA: 1:01:56 - loss: 0.5503 - accuracy: 0.9478

In [ ]:
y_predict = np.argmax(model.predict([x_lr_test,x_hr_test]), axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)